# Tutorial goal

This tutorial aims to show how to **configure** a **test environment** using
the **TestEnv module** provided by LISA.

# Configure logging

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2017-07-14 10:49:29,625 INFO    : root         : Using LISA logging configuration:
2017-07-14 10:49:29,630 INFO    : root         :   /data/lisa_remote/lisa/logging.conf


In [2]:
# Execute this cell to enabled devlib debugging statements
logging.getLogger().setLevel(logging.DEBUG)

In [3]:
# Other python modules required by this notebook
import json
import time
import os

# Test environment setup

## Do you have custom scripts to deploy and use on target?

In [4]:
# Custom scrips must be deployed under $LISA_HOME/tools
!tree ../../../tools

../../../tools
├── arm64
│   ├── perf
│   ├── README.rt-app
│   ├── rt-app
│   ├── sysbench
│   ├── taskset
│   └── trace-cmd
├── armeabi
│   ├── perf
│   ├── README.rt-app
│   ├── rt-app
│   ├── sysbench
│   ├── taskset
│   └── trace-cmd
├── LICENSE.perf
├── LICENSE.rt-app
├── LICENSE.sysbench
├── LICENSE.taskset
├── LICENSE.trace-cmd
├── plots.py
├── report.py
├── scripts
│   ├── cgroup_run_into.sh
│   ├── cgroup_tasks_move.sh
│   ├── cpuidle_sampling.sh
│   ├── odroid_sampler.py
│   ├── odroid_sampler.sh
│   └── trace_frequencies.sh
└── x86_64
    ├── perf
    ├── rt-app
    ├── sysbench
    ├── taskset
    └── trace-cmd

4 directories, 30 files


In [5]:
# This is the (not so fancy) script we want to deploy
!cat ../../../tools/scripts/cpuidle_sampling.sh

#!/system/bin/sh

# CPU to monitor
CPU=${1:-0}
# Sampling time
SLEEP=${2:-1}
# Samples to collect
COUNT=${3:-3}

# Enter CPU's sysfs
cd /sys/devices/system/cpu

# Initial C-State residencies counter
ISC=$(find cpu0/cpuidle -name "state*" | wc -l)
for I in $(seq 0 $((ISC-1))); do
	LCS[$I]=`cat cpu$CPU/cpuidle/state$I/usage`
done

# Dump header
printf "#%13s " "Time"
for I in $(seq 0 $((ISC-1))); do
  printf "%14s " "idle$I"
done
echo

# Sampling loop
for I in $(seq $COUNT); do

	sleep $SLEEP

	# Dump CPU C-State residencies
	now=$(date +%s)
	printf "%14d " $now
	for I in $(seq 0 $((ISC-1))); do
		U=`cat cpu$CPU/cpuidle/state$I/usage`
		CCS=$(($U - ${LCS[$I]}))
		printf "%14d " $CCS
		LCS[$I]=$U
	done
	echo


done

# vim: ts=2


## Which devlib modules you need for your experiments?

In [6]:
# You can have a look at the devlib supported modules by lising the
devlib_modules_folder = 'libs/devlib/devlib/module/'

logging.info("Devlib provided modules are found under:")
logging.info("   $LISA_HOME/{}".format(devlib_modules_folder))
!cd ../../../ ; find {devlib_modules_folder} -name "*.py" | sed 's|libs/devlib/devlib/module/|   |' | grep -v __init__ 

2017-07-14 10:49:32,466 INFO    : root         : Devlib provided modules are found under:
2017-07-14 10:49:32,468 INFO    : root         :    $LISA_HOME/libs/devlib/devlib/module/


   thermal.py
   hwmon.py
   cgroups.py
   hotplug.py
   cpuidle.py
   cooling.py
   biglittle.py
   android.py
   vexpress.py
   gem5stats.py
   cpufreq.py


## Setup you TestEnv confguration

In [7]:
# Setup a target configuration
conf = {

    # Define the kind of target platform to use for the experiments
    "platform"    : 'android',  # platform type, valid other options are:
                              # android - access via ADB
                              # linux   - access via SSH
                              # host    - direct access
    
    # Preload settings for a specific target
    "board"       : 'hikey960',   # board type, valid options are:
                              # - juno  - JUNO Development Board
                              # - tc2   - TC2 Development Board
                              # - hikey - Hikey 8 Cortex-A53 CPU board
                              # - hikey960 - Hikey960 4 Cortex-A73 + 4 Cortex-A53 CPU board

    # Login credentials
    #"host"        : "192.168.0.1",
    "username"    : "root",
    "password"    : "",

    # Custom tools to deploy on target, they must be placed under:
    #   $LISA_HOME/tools/(ARCH|scripts)
    "tools" : [ "cpuidle_sampling.sh" ],

    # FTrace configuration
    "ftrace" : {
         "events" : [
             "cpu_idle",
             "sched_switch",
         ],
         "buffsize" : 10240,
    },
    
    # Where results are collected
    "results_dir" : "TestEnvExample_hikey960",
    
    # Devlib module required (or not required)
    'modules' : [ "cpufreq", "cgroups" ],
    #"exclude_modules" : [ "hwmon" ],
    
    # Local installation path used for kernel/dtb installation on target
    # The specified path MUST be accessible from the board, e.g.
    # - JUNO/TC2: it can be the mount path of the VMESD disk image
    # - Other board: it can be a TFTP server path used by the board bootloader
    "tftp"  : {
        "folder"    : "/var/lib/tftpboot",
        "kernel"    : "kern.bin",
        "dtb"       : "dtb.bin",
    },

}

In [8]:
from env import TestEnv

# Initialize a test environment using the provided configuration
te = TestEnv(conf)

2017-07-14 10:49:34,694 WARNING : EnergyModel  : Unusual max capacity (1023), overriding capacity_scale
2017-07-14 10:49:34,702 INFO    : TestEnv      : Using base path: /data/lisa_remote/lisa
2017-07-14 10:49:34,709 INFO    : TestEnv      : Loading custom (inline) target configuration
2017-07-14 10:49:34,710 INFO    : TestEnv      : External tools using:
2017-07-14 10:49:34,712 INFO    : TestEnv      :    ANDROID_HOME: /home/zhifei/zyang/android-sdk-linux
2017-07-14 10:49:34,713 INFO    : TestEnv      :    CATAPULT_HOME: /data/lisa_remote/lisa/tools/catapult
2017-07-14 10:49:34,718 INFO    : TestEnv      : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq', 'cgroups']
2017-07-14 10:49:34,720 INFO    : TestEnv      : Connecting Android target [DEFAULT]
2017-07-14 10:49:34,722 INFO    : TestEnv      : Connection settings:
2017-07-14 10:49:34,726 INFO    : TestEnv      :    None
2017-07-14 10:49:35,527 INFO    : TestEnv      : Initializing target workdir:
2017-07-14 10:49:35,532 INFO   

## Attributes

The initialization of the test environment pre-initialize some useful<br>
environment variables which are available to write test cases.

These are some of the information available via the TestEnv object.

In [9]:
# The complete configuration of the target we have configured
print json.dumps(te.conf, indent=4)

{
    "username": "root", 
    "platform": "android", 
    "ftrace": {
        "buffsize": 10240, 
        "events": [
            "cpu_idle", 
            "sched_switch"
        ]
    }, 
    "board": "hikey960", 
    "__features__": [], 
    "modules": [
        "cpufreq", 
        "cgroups"
    ], 
    "tftp": {
        "kernel": "kern.bin", 
        "folder": "/var/lib/tftpboot", 
        "dtb": "dtb.bin"
    }, 
    "password": "", 
    "tools": [
        "cpuidle_sampling.sh"
    ], 
    "results_dir": "TestEnvExample_hikey960"
}


In [10]:
# Last configured kernel and DTB image
print te.kernel
print te.dtb

None
None


In [11]:
# The IP and MAC address of the target
print te.ip
print te.mac

None
None


In [12]:
# A full platform descriptor
print json.dumps(te.platform, indent=4)

{
    "kernel": {
        "version_number": 4, 
        "sha1": "679a543", 
        "major": 4, 
        "parts": [
            4, 
            4, 
            59
        ], 
        "version": "2507 SMP PREEMPT Tue Apr 11 09:52:10 PDT 2017", 
        "rc": null, 
        "release": "4.4.59-02536-g679a543", 
        "minor": 59
    }, 
    "abi": "arm64", 
    "freqs": {
        "big": [
            903000, 
            1421000, 
            1805000, 
            2112000, 
            2362000
        ], 
        "little": [
            533000, 
            999000, 
            1402000, 
            1709000, 
            1844000
        ]
    }, 
    "nrg_model": {
        "big": {
            "cluster": {
                "nrg_max": 118
            }, 
            "cpu": {
                "cap_max": 1024, 
                "nrg_max": 1715
            }
        }, 
        "little": {
            "cluster": {
                "nrg_max": 70
            }, 
            "cpu": {
             

In [13]:
# A pre-created folder to host the tests results generated using this
# test environment, notice that the suite could add additional information
# in this folder, like for example a copy of the target configuration
# and other target specific collected information
te.res_dir

'/data/lisa_remote/lisa/results/TestEnvExample_hikey960'

In [14]:
# The working directory on the target
te.workdir

'/data/local/schedtest'

## Functions

Some methods are also exposed to test developers which could be used to easy
the creation of tests.

These are some of the methods available:

In [15]:
# Calibrate RT-App (if required) and get the most updated calibration value
te.calibration()

In [16]:
# Generate a JSON file with the complete platform description
te.platform_dump(dest_dir='/tmp')

({'abi': 'arm64',
  'clusters': {'big': [4, 5, 6, 7], 'little': [0, 1, 2, 3]},
  'cpus_count': 8,
  'freqs': {'big': [903000, 1421000, 1805000, 2112000, 2362000],
   'little': [533000, 999000, 1402000, 1709000, 1844000]},
  'kernel': {'major': 4,
   'minor': 59,
   'parts': (4, 4, 59),
   'rc': None,
   'release': '4.4.59-02536-g679a543',
   'sha1': '679a543',
   'version': '2507 SMP PREEMPT Tue Apr 11 09:52:10 PDT 2017',
   'version_number': 4},
  'nrg_model': {u'big': {u'cluster': {u'nrg_max': 118},
    u'cpu': {u'cap_max': 1024, u'nrg_max': 1715}},
   u'little': {u'cluster': {u'nrg_max': 70},
    u'cpu': {u'cap_max': 870, u'nrg_max': 1210}}},
  'os': 'android',
  'topology': [[0, 1, 2, 3], [4, 5, 6, 7]]},
 '/tmp/platform.json')

In [17]:
# Force a reboot of the target (and wait specified [s] before reconnect)
# te.reboot(reboot_time=60, ping_time=15)

In [18]:
# Resolve a MAC address into an IP address
# te.resolv_host(host='00:02:F7:00:5A:5B')

In [19]:
# Copy the specified file into the TFTP server folder defined by configuration
#te.tftp_deploy('/etc/group')

In [20]:
#!ls -la /var/lib/tftpboot

# Access to the devlib API

A special TestEnv attribute is <b>target</b>, which represents a <b>devlib instance</b>.
Using the target attribute we can access to the full set of devlib provided
functionalities. Which are summarized in the following sections.

## Remotes commands execution

In [21]:
# Run a command on the target
te.target.execute("echo -n 'Hello Test Environment'", as_root=False)

'Hello Test Environment'

In [22]:
# Spawn a command in background on the target
logging.info("Spawn a task which will run for a while...")
process = te.target.kick_off("sleep 10", as_root=True)

2017-07-14 10:50:25,806 INFO    : root         : Spawn a task which will run for a while...
2017-07-14 10:50:26,823 DEBUG   : check_output : 16001 timed out; sending SIGKILL


In [23]:
output = te.target.execute("ps")
print '\n'.join(output.splitlines())

USER           PID  PPID     VSZ    RSS WCHAN            ADDR S NAME                       
root             1     0   13692   5584 0                   0 S init
root             2     0       0      0 0                   0 S [kthreadd]
root             3     2       0      0 0                   0 S [ksoftirqd/0]
root             5     2       0      0 0                   0 S [kworker/0:0H]
root             7     2       0      0 0                   0 S [rcu_preempt]
root             8     2       0      0 0                   0 S [rcu_sched]
root             9     2       0      0 0                   0 S [rcu_bh]
root            10     2       0      0 0                   0 S [migration/0]
root            11     2       0      0 0                   0 S [watchdog/0]
root            12     2       0      0 0                   0 S [watchdog/1]
root            13     2       0      0 0                   0 S [migration/1]
root            14     2       0      0 0                   0 S [ksoft

Notice that _the Shell PID is always the same_ for all commands we execute.<br>
This is due to devlib ensuring to keep a **persistent connection** with the target device.

## Running custom scripts

In [24]:
my_script = te.target.get_installed("cpuidle_sampling.sh")
print my_script

/data/local/tmp/bin/cpuidle_sampling.sh


In [25]:
output = te.target.execute(my_script, as_root=True)
output.splitlines()

['#         Time          idle0          idle1 ',
 '    1500000636             12             30 ',
 '    1500000638             28             29 ',
 '    1500000639             12             29 ']

Notice that the output is returned as a **list of lines**. This provides a useful base for post-processing the output of that command.

In [26]:
# We can also use "notebook embedded" scripts
# my_script = " \
# for I in $(seq 3); do \
#     grep '' /sys/devices/system/cpu/cpu*/cpufreq/stats/time_in_stats | \
#     sed -e 's|/sys/devices/system/cpu/cpu||' -e 's|/cpufreq/scaling_governor:| |' \
#     sleep 1 \
# done \
# "

In [27]:
# print te.target.execute(my_script)

## Access to target specific attributes

In [28]:
# Acces to many target specific information
print "ABI                 : ", te.target.abi
print "big Core Family     : ", te.target.big_core
print "LITTLE Core Family  : ", te.target.little_core
print "CPU's Clusters IDs  : ", te.target.core_clusters
print "CPUs type           : ", te.target.core_names

ABI                 :  arm64
big Core Family     :  A73
LITTLE Core Family  :  A53
CPU's Clusters IDs  :  [0, 0, 0, 0, 1, 1, 1, 1]
CPUs type           :  ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']


In [31]:
# Access to big.LITTLE specific information

print "LITTLE CPUs IDs     : ", te.target.bl.littles
print "LITTLE CPUs freqs      : {}".format(te.target.bl.get_littles_frequency())
print "LITTLE CPUs governor   : {}\n".format(te.target.bl.get_littles_governor())

print "big CPUs IDs        : ", te.target.bl.bigs
print "big CPUs freqs      : {}".format(te.target.bl.get_bigs_frequency())
print "big CPUs governor   : {}".format(te.target.bl.get_bigs_governor())

LITTLE CPUs IDs     :  [0, 1, 2, 3]
LITTLE CPUs freqs      : 533000
LITTLE CPUs governor   : sched

big CPUs IDs        :  [4, 5, 6, 7]
big CPUs freqs      : 903000
big CPUs governor   : sched


## Modules usage example: CPUFreq

In [38]:
# You can use autocompletion to have a look at the supported method for a
# specific module
te.target.cpufreq.get_all_governors()

{'0': 'ondemand',
 '1': 'ondemand',
 '2': 'ondemand',
 '3': 'ondemand',
 '4': 'sched',
 '5': 'sched',
 '6': 'sched',
 '7': 'sched'}

In [34]:
# Get goverors available for CPU0
te.target.cpufreq.list_governors(0)

['interactive', 'ondemand', 'userspace', 'performance', 'sched']

In [35]:
# Set the "ondemand" governor
te.target.cpufreq.set_governor(0, 'ondemand')

In [36]:
# Check governor tunables
te.target.cpufreq.get_governor_tunables(0)

{'ignore_nice_load': '0',
 'io_is_busy': '0',
 'powersave_bias': '0',
 'sampling_down_factor': '1',
 'sampling_rate': '300000',
 'sampling_rate_min': '10000',
 'up_threshold': '95'}

In [39]:
# Update governor tunables
te.target.cpufreq.set_governor_tunables(0, sampling_rate=2000000)
te.target.cpufreq.get_governor_tunables(0)

{'ignore_nice_load': '0',
 'io_is_busy': '0',
 'powersave_bias': '0',
 'sampling_down_factor': '1',
 'sampling_rate': '2000000',
 'sampling_rate_min': '10000',
 'up_threshold': '95'}

## Modules usage example: CGroups

In [41]:
logging.info('%14s - Available controllers:', 'CGroup')
ssys = te.target.cgroups.list_subsystems()
for (n,h,g,e) in ssys:
    print '{:10} (hierarchy id: {:d}) has {} cgroups'.format(n, h, g)

2017-07-14 10:54:13,559 INFO    : root         :         CGroup - Available controllers:


cpuset     (hierarchy id: 5) has 6 cgroups
cpu        (hierarchy id: 4) has 1 cgroups
cpuacct    (hierarchy id: 1) has 84 cgroups
schedtune  (hierarchy id: 2) has 4 cgroups
blkio      (hierarchy id: 6) has 1 cgroups
memory     (hierarchy id: 3) has 2 cgroups
devices    (hierarchy id: 6) has 1 cgroups
freezer    (hierarchy id: 6) has 1 cgroups
hugetlb    (hierarchy id: 6) has 1 cgroups
debug      (hierarchy id: 6) has 1 cgroups


In [42]:
# Get a reference to the CPUSet controller
cpuset = te.target.cgroups.controller('cpuset')

In [43]:
# Get the list of current configured CGroups for that controller
cgroups = cpuset.list_all()
print 'Existing CGropups:'
for cg in cgroups:
    print "   ", cg

2017-07-14 10:54:28,328 DEBUG   : CGroup.cpuset : Listing groups for cpuset controller
2017-07-14 10:54:28,447 DEBUG   : CGroup.cpuset : Populate cpuset cgroup: /
2017-07-14 10:54:28,449 DEBUG   : CGroup.cpuset : Populate cpuset cgroup: /system-background
2017-07-14 10:54:28,450 DEBUG   : CGroup.cpuset : Populate cpuset cgroup: /background
2017-07-14 10:54:28,451 DEBUG   : CGroup.cpuset : Populate cpuset cgroup: /foreground
2017-07-14 10:54:28,455 DEBUG   : CGroup.cpuset : Populate cpuset cgroup: /foreground/boost
2017-07-14 10:54:28,456 DEBUG   : CGroup.cpuset : Populate cpuset cgroup: /top-app


Existing CGropups:
    /
    /system-background
    /background
    /foreground
    /foreground/boost
    /top-app


In [44]:
# Create a LITTLE partition and check which tunables we have
cpuset_littles = cpuset.cgroup('/LITTLE')
cpuset_littles.get()

2017-07-14 10:54:32,433 DEBUG   : cgroups.cpuset : Creating cgroup /data/local/tmp/devlib-target/cgroups/devlib_cgh5/LITTLE
2017-07-14 10:54:32,518 DEBUG   : root         : Reading cpuset attributes from:
2017-07-14 10:54:32,520 DEBUG   : root         :   /data/local/tmp/devlib-target/cgroups/devlib_cgh5/LITTLE


{'cpu_exclusive': '0',
 'cpus': '',
 'effective_cpus': '',
 'effective_mems': '',
 'ls': ' /data/local/tmp/devlib-target/cgroups/devlib_cgh5/LITTLE/cpuset.*',
 'mem_exclusive': '0',
 'mem_hardwall': '0',
 'memory_migrate': '0',
 'memory_pressure': '0',
 'memory_spread_page': '0',
 'memory_spread_slab': '0',
 'mems': '',
 'notify_on_release': '0',
 'sched_load_balance': '1',
 'sched_relax_domain_level': '-1'}

In [45]:
# Setup CPUs and MEMORY nodes for the LITTLE partition
cpuset_littles.set(cpus=te.target.bl.littles, mems=0)

2017-07-14 10:54:37,358 DEBUG   : cgroups.cpuset : Set attribute [/data/local/tmp/devlib-target/cgroups/devlib_cgh5/LITTLE/cpus] to: 0-3"
2017-07-14 10:54:37,539 DEBUG   : cgroups.cpuset : Set attribute [/data/local/tmp/devlib-target/cgroups/devlib_cgh5/LITTLE/mems] to: 0"


In [46]:
# Dump the configuraiton of each controller
for cgname in cgroups:
    cgroup = cpuset.cgroup(cgname)
    attrs = cgroup.get()
    cpus = attrs['cpus']
    print '{}:{:<15} cpus: {}'.format(cpuset.kind, cgroup.name, cpus)

2017-07-14 10:54:41,164 DEBUG   : root         : Reading cpuset attributes from:
2017-07-14 10:54:41,167 DEBUG   : root         :   /data/local/tmp/devlib-target/cgroups/devlib_cgh5
2017-07-14 10:54:41,315 DEBUG   : cgroups.cpuset : Creating cgroup /data/local/tmp/devlib-target/cgroups/devlib_cgh5/system-background
2017-07-14 10:54:41,397 DEBUG   : root         : Reading cpuset attributes from:
2017-07-14 10:54:41,399 DEBUG   : root         :   /data/local/tmp/devlib-target/cgroups/devlib_cgh5/system-background


cpuset:/               cpus: 0-7


2017-07-14 10:54:41,540 DEBUG   : cgroups.cpuset : Creating cgroup /data/local/tmp/devlib-target/cgroups/devlib_cgh5/background
2017-07-14 10:54:41,624 DEBUG   : root         : Reading cpuset attributes from:
2017-07-14 10:54:41,626 DEBUG   : root         :   /data/local/tmp/devlib-target/cgroups/devlib_cgh5/background


cpuset:/system-background cpus: 0-3


2017-07-14 10:54:41,768 DEBUG   : cgroups.cpuset : Creating cgroup /data/local/tmp/devlib-target/cgroups/devlib_cgh5/foreground
2017-07-14 10:54:41,853 DEBUG   : root         : Reading cpuset attributes from:
2017-07-14 10:54:41,855 DEBUG   : root         :   /data/local/tmp/devlib-target/cgroups/devlib_cgh5/foreground


cpuset:/background     cpus: 0


2017-07-14 10:54:41,994 DEBUG   : cgroups.cpuset : Creating cgroup /data/local/tmp/devlib-target/cgroups/devlib_cgh5/foreground/boost
2017-07-14 10:54:42,083 DEBUG   : root         : Reading cpuset attributes from:
2017-07-14 10:54:42,085 DEBUG   : root         :   /data/local/tmp/devlib-target/cgroups/devlib_cgh5/foreground/boost


cpuset:/foreground     cpus: 0-6


2017-07-14 10:54:42,231 DEBUG   : cgroups.cpuset : Creating cgroup /data/local/tmp/devlib-target/cgroups/devlib_cgh5/top-app
2017-07-14 10:54:42,332 DEBUG   : root         : Reading cpuset attributes from:
2017-07-14 10:54:42,334 DEBUG   : root         :   /data/local/tmp/devlib-target/cgroups/devlib_cgh5/top-app


cpuset:/foreground/boost cpus: 0
cpuset:/top-app        cpus: 0-7


In [47]:
# Methods exists to move tasks in/out and in between groups
# cpuset_littles.add_task()

# Sample energy from the target

In [48]:
# If you want to execute this cell successfully, you must connect AEP or ACME to Hikey960

# Reset and sample energy counters
te.emeter.reset()

# Sleep some time
time.sleep(2)

# Sample energy consumption since last reset
nrg = te.emeter.sample()
nrg = json.dumps(te.emeter.sample(), indent=4)
print "First read: ", nrg

# Sleep some more time
time.sleep(2)

# Sample again
nrg = te.emeter.sample()
nrg = json.dumps(te.emeter.sample(), indent=4)
print "Second read: ", nrg

AttributeError: 'NoneType' object has no attribute 'reset'

# Configure FTrace for a sepcific experiment

In [49]:
# Configure a specific set of events to trace
te.ftrace_conf(
    {                                                                                                                                             
         "events" : [                                                                                                                                            
             "cpu_idle",                                                                                                                                         
             "cpu_capacity",
             "cpu_frequency",
             "sched_switch",
         ],                                                                                                                                                      
         "buffsize" : 10240                                                                                                                                      
    }
)

2017-07-14 10:54:57,930 INFO    : TestEnv      : Enabled tracepoints:
2017-07-14 10:54:57,936 INFO    : TestEnv      :    cpu_idle
2017-07-14 10:54:57,938 INFO    : TestEnv      :    cpu_capacity
2017-07-14 10:54:57,939 INFO    : TestEnv      :    cpu_frequency
2017-07-14 10:54:57,943 INFO    : TestEnv      :    sched_switch


In [8]:
# Start/Stop a FTrace session
te.ftrace.start()
te.target.execute("uname -a")
te.ftrace.stop()

02:59:45  DEBUG   : sudo -- sh -c 'echo 10240 > '\''/sys/kernel/debug/tracing/buffer_size_kb'\'''
02:59:45  DEBUG   : sudo -- sh -c 'cat '\''/sys/kernel/debug/tracing/buffer_size_kb'\'''
02:59:45  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd reset'
02:59:46  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd start -e cpu_idle -e cpu_capacity -e cpu_frequency -e sched_switch'
02:59:48  DEBUG   : sudo -- sh -c 'echo TRACE_MARKER_START > '\''/sys/kernel/debug/tracing/trace_marker'\'''
02:59:48  DEBUG   : Trace CPUFreq frequencies
02:59:48  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/shutils cpufreq_trace_all_frequencies'
02:59:48  DEBUG   : uname -a
02:59:49  DEBUG   : Trace CPUFreq frequencies
02:59:49  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/shutils cpufreq_trace_all_frequencies'
02:59:49  DEBUG   : sudo -- sh -c 'echo TRACE_MARKER_STOP > '\''/sys/kernel/debug/tracing/trace_marker'\'''
02:59:49  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-c

In [9]:
# Collect and visualize the trace
trace_file = os.path.join(te.res_dir, 'trace.dat')
te.ftrace.get_trace(trace_file)
output = os.popen("DISPLAY=:0.0 kernelshark {}".format(trace_file))

02:59:50  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd extract -o /root/devlib-target/trace.dat'
02:59:51  DEBUG   : /usr/bin/scp -r   root@192.168.0.1:/root/devlib-target/trace.dat /home/derkling/Code/lisa/results/TestEnvExample/trace.dat
